# Content and Objective

+ Demonstrate effect of low-pass on a rectangular signal

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy import signal

# Reminder
+ The convolution of two sequences is defined as:
  $$(x*h)[n]:=\sum_{k=-\infty}^{\infty} h[k]\cdot x[n-k]$$

# Effects of Impulse Response and Low-pass Filtering

### Implementation of Non-Return-To-Zero modulation:

+ An binary input value is simply repeated as often as specified  
+ Similar to sample-and-hold

In [2]:
def nrz( binary_data, samples_per_bit ):
    '''
    IN: Binary bit stream, Over sampling factor

    OUT: NRZ modulated sequence
    '''
    return np.repeat( binary_data, samples_per_bit )


### Influence of a Low-Pass System on a modulated signal

 In the next cells we investigate the influence of a channel impulse response on our modulated signal. To do so, we modulate our bit stream using NRZ modulation. Afterwards, we transmit the signal over a channel with low-pass behavior. In a simple case, a channel with low-pass behavior can be modeled by an RC element. 

**NOTE:** An RC element has the (continuous-time) transfer function
$$G(s)=\frac{1}{1+RCs}$$
 with
    $$ \tau=RC $$
   being the time constant. The higher $\tau$ the lower the cutoff frequency. Our signal consists of high frequency components, due to the NRZ modulation. Therefore, we expect a stronger influence on our signal, the lower the cutoff frequency gets, because high frequency are strongly attenuated.



In [3]:
def rc_impulse_response( time_constant, time):
    '''
    use on-board methods for defining the transfer function of an RC element and the resulting impulse response

    IN: Time constant and discretization of time

    OUT: sampled values of the impulse response
    '''

    rc_element = signal.TransferFunction( [1] , [time_constant, 1])
    
    return signal.impulse( rc_element, T = time )




### Generate bitstream and modulate it with NRZ

In [4]:
#Define time and discretize it
bit_length = 20
samples_per_bit = 100

time = np.arange( 0, bit_length, step = 1/samples_per_bit )

#Randomly generate data
binary_data = np.random.randint( low=2, size = bit_length )  # bit per second

##Non-Return-to-Zero (NRZ) Modulation of the binary data
nrz_signal = nrz( binary_data, samples_per_bit ) 




### Define plotting function that allows to adapt the time constant

In [5]:
def plot_convol( time_constant ):
    '''
    IN: Time_constant
    OUT: 2 Plots.
    '''

    #Sampled impulse response of the rc element
    t, impulse_response_rc = rc_impulse_response( time_constant, time )

    #Signal after transmission over the channel modeled by an RC element.
    convolved = signal.convolve( nrz_signal, impulse_response_rc )
    convolved = convolved / np.max( np.abs( convolved ) )

    
    #PLOTTING
    fig = plt.figure(figsize=(25, 10))


    # First plot 
    ax = plt.subplot(2, 1, 1)
    
    # Modulated signal      
    plt.plot(time, nrz_signal, label='NRZ modulated signal x[n]', c='k')
    
    # Impulse Response of the RC element
    plt.plot(t, impulse_response_rc/np.max( np.abs( impulse_response_rc ) ),
             label='Impulse response h[n]' )

    # Parametrize plot
    ax.set_xlabel('t in [s]')    
    plt.legend()#bbox_to_anchor=(1, 1.05))
    ax.set_title('NRZ modulated signal and impulse response of the RC element')
    plt.xlim(0, bit_length+2)
    

    # Second plot
    ax = plt.subplot(2, 1, 2)


    # Plot original NRZ sequence as reference
    plt.plot(time, nrz_signal, label='NRZ modulated signal x[n]', c='k')

    # Plot of the signal after transmission
    plt.plot(np.arange(start=0, stop=0.01*(len(convolved)),
                       step=0.01), convolved, label='x[n] * h[n]')
    
    # Parametrize plot
    ax.set_title(
        'Original signal and signal after convolution with RC element')
    plt.xlim(0, bit_length+2)
    ax.set_title('C')
    ax.set_xlabel('t in [s]')
    ax.legend( )#bbox_to_anchor=(1, 1.05))
    plt.tight_layout()


### Plotting

+ First plot: 
    + Impulse response of the RC element in blue: $h[n]$ 
    + NRZ modulated and upsampled signal in black $x[n]$
    + The slider allows to vary the time constant $\tau$.
+ Second plot: 
    + Show effect of filtering.
    + For higher time constants $\tau$ the sharp edges of the NRZ modulation get smoothened such that the different bits can not be distinguished anymore. 
    + This effect is called Inter-Symbol-Interference (ISI) and leads to undesired behavior. Thus, the phenomenon must be avoided using, e.g., guard intervals in between the bit positions.

In [6]:
w = widgets.interact(plot_convol, time_constant=widgets.FloatSlider(
    min=0.001, max=2, step=0.01, continuous_update=True, value=0.1))


interactive(children=(FloatSlider(value=0.1, description='time_constant', max=2.0, min=0.001, step=0.01), Outp…